# Pràctica 2: Recomanador Simple

Nom dels alumnes del grup: Mohsin Zaman y Raquel


## 1. INTRODUCCIÓ

### 1.1. Abans de començar...

**\+ A més a més de les que ja es troben presents en la 1a cel·la i funcions natives de Python, durant la pràctica, només es podran fer servir les següents llibreries**:

`Pandas, Numpy, Itertools`

**\+ No es poden modificar les definicions de les funcions donades, ni canviar els noms de les variables i paràmetres ja donats**

Això no implica però que els hàgiu de fer servir. És a dir, que la funció tingui un paràmetre anomenat `df` no implica que l'hàgiu de fer servir, si no ho trobeu convenient.

**\+ En les funcions, s'especifica què serà i de quin tipus cada un dels paràmetres, cal respectar-ho**

Per exemple (ho posarà en el pydoc de la funció), `df` sempre serà indicatiu del `Pandas.DataFrame` de les dades.

### 1.2. Dades: puntuacions de pel·licules

La base de dades [movielens-1M](http://www.grouplens.org/node/73) conté 1,000,209 puntuacions de 3.900 pel·lícules fetes l'any 2000 per 6.040 usuaris anònims del recomanador online [MovieLens](http://www.movielens.org/). 

El consum total de tots els usuaris s'hi pot trobar al document "ratings.dat" el format següent:

    UserID::MovieID::Rating::Timestamp

- **UserID** usuari, amb id's entre 1 i 6040 
- **MovieID** pel·licula, amb id's entre 1 i 3952
- **Rating** puntuació, en una escala de 1 a 5 estrelles.
- **Timestamp** representat en segons

Cada usuari té com a mínim 20 puntuacions.

.3### 1.3. Dades: usuaris



Al fitxer ``users.dat`` hi trobem la informació referent a cadascun dels usuaris en el següent format:

        UserID::Gender::Age::Occupation::Zip-code

- **Gender** ve donat per "M" per home i "F" per dona.
- **Age** està representada de la següent forma:

	*  1:  "Under 18"
	* 18:  "18-24"
	* 25:  "25-34"
	* 35:  "35-44"
	* 45:  "45-49"
	* 50:  "50-55"
	* 56:  "56+"

- **Occupation** es tria entre les següents opcions:

	*  0:  "other" or not specified
	*  1:  "academic/educator"
	*  2:  "artist"
	*  3:  "clerical/admin"
	*  4:  "college/grad student"
	*  5:  "customer service"
	*  6:  "doctor/health care"
	*  7:  "executive/managerial"
	*  8:  "farmer"
	*  9:  "homemaker"
	* 10:  "K-12 student"
	* 11:  "lawyer"
	* 12:  "programmer"
	* 13:  "retired"
	* 14:  "sales/marketing"
	* 15:  "scientist"
	* 16:  "self-employed"
	* 17:  "technician/engineer"
	* 18:  "tradesman/craftsman"
	* 19:  "unemployed"
	* 20:  "writer"

Els usuaris han donat la informació voluntariament. Així doncs, alguns usuaris poden no tenir informació.


### 1.4. Dades: pel·lícules



Al fitxer ``movies.dat`` hi trobem la informació referent a cadascuna de les películes en el següent format:

        MovieID::Title::Genres

- **Titles** són identics als titols de la base de dades IMDB, incloent l'any de llançament.
- **Genres** de les películes estan separats i seleccionats d'entre els següents:

	* Action
	* Adventure
	* Animation
	* Children's
	* Comedy
	* Crime
	* Documentary
	* Drama
	* Fantasy
	* Film-Noir
	* Horror
	* Musical
	* Mystery
	* Romance
	* Sci-Fi
	* Thriller
	* War
	* Western

Algunes películes poden tenir l'ID malament degut a duplicats accidentals.

Les películes s'han entrat manualment, així que poden existir altres inconsistencies. 

## 2. Exploració de les dades

### 2.1. Descarregar i llegir dades

+ Baixa't els fitxers que composen la base de dades i els còpies al teu directori de treball. 

In [95]:
import os
if os.path.isfile("/etc/password.txt") == False:
    os.system('wget -nc http://files.grouplens.org/datasets/movielens/ml-1m.zip')
    os.system('unzip ml-1m.zip')

+ Llegeix les tres taules de la base de dades en tres DataFrames de pandas amb aquest codi:

In [3]:
import math
import numpy as np
import pandas as pd
import datetime
import itertools
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt

In [97]:
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('ml-1m/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('ml-1m/ratings.dat', sep='::', header=None, names=rnames, engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('ml-1m/movies.dat', sep='::', header=None, names=mnames, engine='python', encoding='latin-1')



### 2.2. Inspecció de les taules

In [98]:
users[:10]

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
5,6,F,50,9,55117
6,7,M,35,1,06810
7,8,M,25,12,11413
8,9,M,25,17,61614
9,10,F,35,1,95370


In [99]:
users[-10:]

,user_id,gender,age,occupation,zip
6030,6031,F,18,0,45123
6031,6032,M,45,7,55108
6032,6033,M,50,13,78232
6033,6034,M,25,14,94117
6034,6035,F,25,1,78734
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060
6039,6040,M,25,6,11106


In [100]:
ratings[-10:]

,user_id,movie_id,rating,timestamp
1000199,6040,2022,5,956716207
1000200,6040,2028,5,956704519
1000201,6040,1080,4,957717322
1000202,6040,1089,4,956704996
1000203,6040,1090,3,956715518
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569


In [101]:
ratings[:10]

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [102]:
ratings.sort_values('movie_id')[:5]


,user_id,movie_id,rating,timestamp
427702,2599,1,4,973796689
1966,18,1,4,978154768
683688,4089,1,5,965428947
596207,3626,1,4,966594018
465902,2873,1,5,972784317


In [103]:
movies[0:]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [104]:
ratings[:5]

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


### 2.3 **Exemple:** Com extreure informació d'un DataFrame.

Suposa que volem calcular les **puntuacions mitjanes d'una pel·licula per sexe o edat**, dades que estan a frames diferents.

El primer pas a obtenir una única estructura que contingui tota la informació. Per fer-ho podem usar la funció ``merge`` de pandas. Aquesta funció infereix automàticament quines columnes ha d'usar per fer el ``merge`` basant-se en els noms que fan intersecció.

Reviseu aquests conceptes de pandas: https://pandas.pydata.org/docs/user_guide/merging.html

In [105]:
data = pd.merge(pd.merge(ratings, users), movies)

# Visualitzem la taula ordenada per identificar d'usuari
data.sort_values(by='user_id')[:10]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
28501,1,48,5,978824351,F,1,10,48067,Pocahontas (1995),Animation|Children's|Musical|Romance
13819,1,938,4,978301752,F,1,10,48067,Gigi (1958),Musical
51327,1,1207,4,978300719,F,1,10,48067,To Kill a Mockingbird (1962),Drama
31152,1,1721,4,978300055,F,1,10,48067,Titanic (1997),Drama|Romance
37916,1,2762,4,978302091,F,1,10,48067,"Sixth Sense, The (1999)",Thriller
18472,1,2687,3,978824268,F,1,10,48067,Tarzan (1999),Animation|Children's
45685,1,2692,4,978301570,F,1,10,48067,Run Lola Run (Lola rennt) (1998),Action|Crime|Romance
22832,1,720,3,978300760,F,1,10,48067,Wallace & Gromit: The Best of Aardman Animatio...,Animation
32771,1,745,3,978824268,F,1,10,48067,"Close Shave, A (1995)",Animation|Comedy|Thriller


La funció ``iloc`` ens permet obtenir un subconjunt de files i/o columnes indexades per un enter:

In [106]:
data.iloc[3:5]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama


Els índexs Booleans ens permeten seleccionar una part de la taula que compleix una condició.

In [107]:
# comptem quin tant per cent de ratings estan fets per una dona

# Del dataframe "data", crea un df con solo gender F. De ese, de la columna 'rating' hace la suma y divide para sacar el porcentaje
print(data[data['gender']=='F']['rating'].count()/float(data['rating'].count())*100, '%')

24.638850480249626 %


Per obtenir les **puntuacions mitjanes de cada pel·licula agrupada per edat** podem usar el mètode ``pivot_table`` que és una forma de "canviar" la forma de la taula especificant quin valor agregat (mitjançant una funció predefinida) hi volem en funció dels valors de dues columnes.

Reviseu aquests conceptes: 
+ https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html
+ https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot_table.html#pandas.DataFrame.pivot_table

In [160]:
mean_ratings = data.pivot_table(values= 'rating', index='title', columns='age', aggfunc='mean')
mean_ratings[:10]

age,1,18,25,35,45,50,56
title,,,,,,,
"$1,000,000 Duck (1971)",NaN,3.000000,3.090909,3.133333,2.000000,2.750000,NaN
'Night Mother (1986),2.000000,4.666667,3.423077,2.904762,3.833333,3.555556,4.333333
'Til There Was You (1997),3.500000,2.500000,2.666667,2.900000,2.333333,2.500000,2.666667
"'burbs, The (1989)",4.500000,3.244444,2.652174,2.818182,2.545455,3.208333,2.666667
...And Justice for All (1979),3.000000,3.428571,3.724138,3.657143,4.100000,3.551724,3.928571
1-900 (1994),NaN,NaN,2.000000,NaN,NaN,NaN,3.000000
10 Things I Hate About You (1999),3.745455,3.415020,3.432950,3.102941,3.258065,3.629630,4.000000
101 Dalmatians (1961),3.514286,3.295082,3.613757,3.826087,3.976744,3.650000,3.190476
101 Dalmatians (1996),3.088235,2.467742,2.928571,3.279570,3.482759,3.400000,3.555556


Per obtenir les **puntuacions mitjanes de cada pel·licula agrupada per sexe**:

In [109]:
mean_ratings = data.pivot_table('rating', index='title',columns='gender', aggfunc='mean')
mean_ratings[:10]

gender,F,M
title,,
"$1,000,000 Duck (1971)",3.375000,2.761905
'Night Mother (1986),3.388889,3.352941
'Til There Was You (1997),2.675676,2.733333
"'burbs, The (1989)",2.793478,2.962085
...And Justice for All (1979),3.828571,3.689024
1-900 (1994),2.000000,3.000000
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215


Si volgéssim fer càlculs només sobre les pel·licules que han rebut **al menys** 250 puntuacions, primer hem de construir una taula amb el nombre d'avaluacions de cada títol. Per fer-ho, agruparem les dades per títol (amb el mètode ``groupby``) i usarem ``size()`` el nombre.

Reviseu aquest concepte: 

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html

El mètode ``groupby`` implenta un o més d'aquests processos:

+ Dividir les dades segons algun criteri.
+ Aplicar una funció a cada grup.
+ Combinar els resultats en una estructura de dades.

In [110]:
ratings_by_title = data.groupby('title').size()
print(ratings_by_title)

title
$1,000,000 Duck (1971)                         37
'Night Mother (1986)                           70
'Til There Was You (1997)                      52
'burbs, The (1989)                            303
...And Justice for All (1979)                 199
                                             ... 
Zed & Two Noughts, A (1985)                    29
Zero Effect (1998)                            301
Zero Kelvin (Kjærlighetens kjøtere) (1995)      2
Zeus and Roxanne (1997)                        23
eXistenZ (1999)                               410
Length: 3706, dtype: int64


Llavors podem crear un índex amb els títols amb més de 250 avaluacions.

In [111]:
active_titles = ratings_by_title.index[ratings_by_title >= 250]
active_titles

Index([''burbs, The (1989)', '10 Things I Hate About You (1999)',
       '101 Dalmatians (1961)', '101 Dalmatians (1996)', '12 Angry Men (1957)',
       '13th Warrior, The (1999)', '2 Days in the Valley (1996)',
       '20,000 Leagues Under the Sea (1954)', '2001: A Space Odyssey (1968)',
       '2010 (1984)',
       ...
       'X-Men (2000)', 'Year of Living Dangerously (1982)',
       'Yellow Submarine (1968)', 'You've Got Mail (1998)',
       'Young Frankenstein (1974)', 'Young Guns (1988)',
       'Young Guns II (1990)', 'Young Sherlock Holmes (1985)',
       'Zero Effect (1998)', 'eXistenZ (1999)'],
      dtype='object', name='title', length=1216)

L'índex de títols que reben al menys 250 puntuacions es pot fer servir per seleccionar les files de ``mean_ratings``: 

In [112]:
mean_ratings = mean_ratings.loc[active_titles]
mean_ratings

gender,F,M
title,,
"'burbs, The (1989)",2.793478,2.962085
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215
12 Angry Men (1957),4.184397,4.328421
...,...,...
Young Guns (1988),3.371795,3.425620
Young Guns II (1990),2.934783,2.904025
Young Sherlock Holmes (1985),3.514706,3.363344


Per veure els films més valorats per les dones, podem ordenar per la columna F de forma descendent:

In [113]:
top_female_ratings = mean_ratings.sort_values(by='F', ascending=False)
top_female_ratings[:10]

gender,F,M
title,,
"Close Shave, A (1995)",4.644444,4.473795
"Wrong Trousers, The (1993)",4.588235,4.478261
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.572650,4.464589
Wallace & Gromit: The Best of Aardman Animation (1996),4.563107,4.385075
Schindler's List (1993),4.562602,4.491415
"Shawshank Redemption, The (1994)",4.539075,4.560625
"Grand Day Out, A (1992)",4.537879,4.293255
To Kill a Mockingbird (1962),4.536667,4.372611
Creature Comforts (1990),4.513889,4.272277


Suposem ara que volem les pel·licules que estan valorades de forma més diferent entre homes i dones. Una forma d'obtenir-ho és afegir una columna a ``mean_ratings`` que contingui la diferència en mitjana i llavors ordenar:

In [114]:
mean_ratings['diff'] = mean_ratings['M'] - mean_ratings['F']

In [115]:
print(np.nan + 9.0) 

nan


Ordenant per ``diff`` ens dóna les pel·licules ben valorades per les dones que presenten més diferència entre homes i dones:

In [116]:
sorted_by_diff = mean_ratings.sort_values(by='diff')
sorted_by_diff[:15]

gender,F,M,diff
title,,,
Dirty Dancing (1987),3.790378,2.959596,-0.830782
Jumpin' Jack Flash (1986),3.254717,2.578358,-0.676359
Grease (1978),3.975265,3.367041,-0.608224
Little Women (1994),3.870588,3.321739,-0.548849
Steel Magnolias (1989),3.901734,3.365957,-0.535777
Anastasia (1997),3.800000,3.281609,-0.518391
"Rocky Horror Picture Show, The (1975)",3.673016,3.160131,-0.512885
"Color Purple, The (1985)",4.158192,3.659341,-0.498851
"Age of Innocence, The (1993)",3.827068,3.339506,-0.487561


Invertint l'ordre de les files i fent un ``slicing`` de les 15 files superiors obtenim les pel·licules ben valorades pels homes que no han agradat a les dones: 

In [117]:
sorted_by_diff[::-1][:15]

gender,F,M,diff
title,,,
"Good, The Bad and The Ugly, The (1966)",3.494949,4.221300,0.726351
"Kentucky Fried Movie, The (1977)",2.878788,3.555147,0.676359
Dumb & Dumber (1994),2.697987,3.336595,0.638608
"Longest Day, The (1962)",3.411765,4.031447,0.619682
"Cable Guy, The (1996)",2.250000,2.863787,0.613787
Evil Dead II (Dead By Dawn) (1987),3.297297,3.909283,0.611985
"Hidden, The (1987)",3.137931,3.745098,0.607167
Rocky III (1982),2.361702,2.943503,0.581801
Caddyshack (1980),3.396135,3.969737,0.573602


Si volguéssim les pel·licules que han generat puntuacions més discordants, independentment del gènere, podem fer servir la variança o la desviació estàndard de les puntuacions: 

In [118]:
# Standard deviation of rating grouped by title
rating_std_by_title = data.groupby('title')['rating'].std()
# Filter down to active_titles
rating_std_by_title = rating_std_by_title.loc[active_titles]
rating_std_by_title.sort_values(ascending=False)[:10]

title
Dumb & Dumber (1994)                     1.321333
Blair Witch Project, The (1999)          1.316368
Natural Born Killers (1994)              1.307198
Tank Girl (1995)                         1.277695
Rocky Horror Picture Show, The (1975)    1.260177
Eyes Wide Shut (1999)                    1.259624
Evita (1996)                             1.253631
Billy Madison (1995)                     1.249970
Fear and Loathing in Las Vegas (1998)    1.246408
Bicentennial Man (1999)                  1.245533
Name: rating, dtype: float64

### Important: Temes de rendiment

Fixeu-vos en el comportament de Python i actueu en conseqüència:

In [119]:
%timeit data['title'] 
print(type(data['title']))
%timeit data.title 
print(type(data.title))
%timeit data[['title']] 
print(type(data[['title']]))

1.19 µs ± 4.43 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
<class 'pandas.core.series.Series'>
2.89 µs ± 15.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
<class 'pandas.core.series.Series'>
8.13 ms ± 72.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
<class 'pandas.core.frame.DataFrame'>


## 3. EXERCICIS

### 3.1. EXERCICI A

+ Donada la taula ``data`` tal i com es defineix a continuació, calcula la puntuació mitjana de cada usuari i guarda-la a un ``df`` anomenat ``users_mean_rating``. 

In [170]:
data_folder = 'ml-1m'

In [171]:
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table(f'{data_folder}/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table(f'{data_folder}/ratings.dat', sep='::', header=None, names=rnames, engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table(f'{data_folder}/movies.dat', sep='::', header=None, names=mnames, engine='python',encoding='latin-1')

data = pd.merge(pd.merge(ratings, users), movies)
users_mean_rating = data.pivot_table('rating', index='user_id', aggfunc='mean')

users_mean_rating

,rating
user_id,
1,4.188679
2,3.713178
3,3.901961
4,4.190476
5,3.146465
...,...
6036,3.302928
6037,3.717822
6038,3.800000


+ Quina és la pel·lícula més ben puntuada (en mitja) pels usuaris? (Guarda aquest valor en una variable de tipus ``string`` anomenada ``best_movie_rating`` ). 

In [172]:
# la vostra solució aquí
movie_mean = data.pivot_table('rating', index='title', aggfunc='mean')
best_movie = movie_mean.loc[movie_mean['rating'].idxmax()]
best_movie_rating = best_movie.name

print(best_movie_rating)

Baby, The (1973)


+ Mira si hi ha més pel·licules amb la mateixa puntuació de la més ben puntuada.

In [173]:
# la vostra solució aquí

otherMoviesWithSameRating = movie_mean.query(f'rating == {best_movie.rating}')
print(otherMoviesWithSameRating)

                                           rating
title                                            
Baby, The (1973)                              5.0
Bittersweet Motel (2000)                      5.0
Follow the Bitch (1998)                       5.0
Gate of Heavenly Peace, The (1995)            5.0
Lured (1947)                                  5.0
One Little Indian (1973)                      5.0
Schlafes Bruder (Brother of Sleep) (1995)     5.0
Smashing Time (1967)                          5.0
Song of Freedom (1936)                        5.0
Ulysses (Ulisse) (1954)                       5.0


+ Busca ara aquella pel·lícula, d'entre les que tenen 5 com a puntuació mitjana, que hagi rebut més valoracions i guarda-la a una variable anomenada ``best_movie_rating_maxviews``. Així tindrem la pel·licula més ben puntuada per més usuaris. 

In [174]:
# la vostra solució aquí

# Busco numero de valoracions de cada pelicula
movie_most_ratings = data.groupby('title').size()

# De aquestes pelicules, filtro nomes les que tenen valoracio mes alta
filtered = movie_most_ratings.loc[otherMoviesWithSameRating.index]

 # busquem la pelicula amb major numero de valoracions
best_movie_rating_maxviews = filtered.idxmax()
print(best_movie_rating_maxviews)



Gate of Heavenly Peace, The (1995)


### 3.2. EXERCICI B

+ Defineix una funció anomenada ``top_movie`` que donat un usuari ens retorni quina és la pel·lícula millor puntuada.


In [175]:
def top_movie(dataFrame,usr):
# la vostra solució aquí

    # primer busquem les valoracions a les pelicules que ha fet aquest usuari
    rated_movies_by_user = dataFrame[dataFrame['user_id'] == usr]
    movie_mean_ratings = rated_movies_by_user.groupby('title')['rating'].mean()
    # Troba la mitjana més alta de aquest usuari
    max_rating_by_this_user = movie_mean_ratings.max()
    # Enlloc de retornar només una película, retornem totes les pelicules amb valoracio més alta de aquest ususari ja que hi ha mes d'una
    # amb valoració de 5
    all_best_ratings = movie_mean_ratings[movie_mean_ratings == max_rating_by_this_user].index.tolist()
    return all_best_ratings
    
    
print(top_movie(data,1))

['Apollo 13 (1995)', 'Awakenings (1990)', 'Back to the Future (1985)', 'Beauty and the Beast (1991)', 'Ben-Hur (1959)', "Bug's Life, A (1998)", 'Christmas Story, A (1983)', 'Cinderella (1950)', 'Dumbo (1941)', 'Last Days of Disco, The (1998)', 'Mary Poppins (1964)', "One Flew Over the Cuckoo's Nest (1975)", 'Pocahontas (1995)', 'Rain Man (1988)', 'Saving Private Ryan (1998)', "Schindler's List (1993)", 'Sound of Music, The (1965)', 'Toy Story (1995)']


### 3.3. EXERCICI C

+ Construeix una funció que donat el dataframe ``data`` et retorni un altre dataframe ``df_counts``amb el valor que cada usuari li ha donat a una peli. Això ho farem creant un dataframe on les columnes són els `movie_id`, les files `user_id` i els valors siguin el rating donat.

In [176]:
def build_counts_table(df):
    """
    Retorna un dataframe on les columnes són els `movie_id`, les files `user_id` i els valors
    la valoració que un usuari ha donat a una peli d'un `movie_id`
    
    :param df: DataFrame original 
    :return: DataFrame descrit adalt
    """
    
    # la vostra solució aquí
    # Poso un valor de NaN a les pelis que el usuari no hagi valorat
    #return df.pivot_table(values='rating', index='user_id', columns='movie_id', fill_value=None)

    return df.pivot_table('rating', index='user_id',columns='movie_id')
    
    

In [177]:
df_counts = build_counts_table(data)
df_counts

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,NaN,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


+ Fés una funció que donada la taula anterior i dos id's (usuari i peli), extregui el valor donat:

In [178]:
def get_count(df, user_id, movie_id):
    """
    Retorna la valoració que l'usuari 'user_id' ha donat de 'movie_id'
    
    :param df: DataFrame retornat per `build_counts_table`
    :param user_id: ID de l'usuari
    :param movie_id: ID de la peli
    :return: Enter amb la valoració de la peli
    """
    
    # la vostra solució aquí

    return df.loc[user_id,movie_id]
    

get_count(df_counts, 1, 1)

5.0

### 3.4. EXERCICI D

In [179]:
data.nunique()

user_id         6040
movie_id        3706
rating             5
timestamp     458455
gender             2
age                7
occupation        21
zip             3439
title           3706
genres           301
dtype: int64

In [180]:
unique_movies = pd.unique(data['movie_id'])
unique_movies.max()

3952

Si observem el nombre total d'usuaris únics i de pel.licules úniques, podem veure que els id's dels usuaris van de 1 a 6040. Normalment volem índexos que comencin al nombre 0, anant de 0 a 6039. 

+ Explora els índexos de les pel·licules. **Quin problema hi ha amb els indexos de les pel·licules??**

> **Resposta**: El problema que hi ha amb els indexos de les pelicules es que el algunes ids de les pelicules no estan en la taula movies i per tant, com el df compença desde index 0 mentre que les ids començen desde 1 llavors si algun id no esta a la taula doncs no hi haura una coincidencia entre el numero de ids i els indexos del dataframe o taula.


+ Usant la funció `pd.Categorical(*).codes`, re-indexa els id's dels usuaris i de les pelis perquè vagin de 0 a 6039 i de 0 a 3705 respectivament:

In [181]:
# la vostra solució aquí

# Reindexem els IDs dels usuaris
data['user_id'] = pd.Categorical(data['user_id']).codes
# Reindexem els IDs de les pel·lícules
data['movie_id'] = pd.Categorical(data['movie_id']).codes



+ Per comprovar que tot sigui correcte i guardar correctament la taula **df_counts**, torna a calcular i visualitza ``df_counts``:

In [182]:
# la vostra solució aquí
df_counts = build_counts_table(data)
df_counts

movie_id,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
user_id,,,,,,,,,,,,,,,,,,,,,
0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,NaN,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 3.5. EXERCICI E



+ Escriu una funció `distEuclid(x,y)`  que implementi la distància Euclidiana entre dos vectors usant funcions de pandas. 

+ Escriu la funció `SimEuclid (DataFrame, U1, U2)` que calculin la semblança entre dos usuaris segons aquesta estructura:

    + Calcular un vector per cada usuari, $U1$ i $U2$, amb les puntuacions dels ítems comuns que han puntuat el dos usuaris.
    + Si no hi ha puntuacions en comú, retornar 0. Sinó, retornar 
    
    $$\frac{1}{(1+distEuclid(U1, U2))}$$

+ Avalueu amb la funció ``%timeit`` quant triguen aquests càlculs per un parell d'usuaris.   

> *Nota: Alguns d'aquests exercicis tenen temps de càlcul de l'ordre de minuts sobre tota la base de dades. Per desenvolupar els algorismes és recomanable treballar amb una versió reduïda de la base de dades.* 

Per implementar aquestes funcions únicament es permet l'ús de les funcions:

* `np.sum`
* `np.sqrt`
* `np.power`
* `np.dot`
* `np.linalg.norm`
* `np.mean`

I s'ha de fer **sense bucles**!

In [183]:
num_movies = data.nunique()['movie_id']
print("num_movies", num_movies)
def distEuclid(x, y):
    """
    Retorna la distancia euclidiana de dos vectors n-dimensionals.

    :param x: Primer vector
    :param y: Segon vector
    :return : Escalar (float) corresponent a la distancia euclidiana
    """

    # la vostra solució aquí

    return np.linalg.norm(x-y)


def SimEuclid(DataFrame, User1, User2):
    """
    Retorna un score que representa la similitud entre user1 i user2 basada en la distancia euclidiana

    :param DataFrame: dataframe que conté totes les dades
    :param User1: id user1
    :param User2: id user2
    :return : Escalar (float) corresponent al score
    """
    # la vostra solució

    # Fem un vector per a cada usuari, amb la llista dels seus ratings
    data_user1 = np.full(num_movies, np.nan)
    user_data = DataFrame.loc[DataFrame['user_id'] == User1, ['movie_id', 'rating']]
    data_user1[user_data['movie_id']] = user_data['rating'].values

    data_user2 = np.full(num_movies, np.nan)
    user_data = DataFrame.loc[DataFrame['user_id'] == User2, ['movie_id', 'rating']]
    data_user2[user_data['movie_id']] = user_data['rating'].values

    # Fem una mascara per filtrar els items que tenen en comú
    mask = np.power(data_user1, 2) == np.power(data_user1, 2)
    mask &= np.power(data_user2, 2) == np.power(data_user2, 2)
    
    # Filtrem els vectors amb la mascara
    filtered_data_user1 = data_user1[mask]
    filtered_data_user2 = data_user2[mask]
    
    # Comprovem si hi ha elements en comú
    if len(filtered_data_user1) == 0:
        return 0
    else:
        return (1/(1+distEuclid(filtered_data_user1, filtered_data_user2)))* (len(filtered_data_user1)/num_movies)

num_movies 3706


In [184]:
# Execute functions
print(SimEuclid(data, 2,314))

0.0005396654074473826


In [185]:
# Execute functions
print(SimEuclid(data, 0,1))

0.0006296096420219464


In [186]:
# Execute functions
print(SimEuclid(data, 1, 5))

0.0005316828473355942


In [187]:
%timeit SimEuclid(data, 1, 5)

4.57 ms ± 493 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### 3.6. EXERCICI F

En aquest exercici desenvoluparem un sistema de recomanació col·laboratiu **basat en usuaris**. 

La funció principal, ``getRecommendationsUser``, ha de tenir com a entrada una taula de puntuacions, un ``user_id``, el tipus de mesura de semblança (Euclidiana) que volem usar, el nombre `m` d'usuaris semblants que volem per fer la recomanació i el nombre ``n`` de recomanacions que volem. 

Exemple: ``getRecommendationsUser(data, 2, 50, 10, SimEuclid)``

Com a sortida ha de donar la llista de les $n$ millors pel·lícules que li podriem recomanar segons la seva semblança amb altres usuaris.

> *Nota 1: S'ha d'evitar comparar ``user_id`` a ell mateix.*

> *Nota 2: Recordeu que en Python podem passar funcions com a paràmetres d'una funció.*

#### EXERCICI F.1

+ Computa la *score* de similitud del usuari desitjat (userID) respecte tots els altres i retorna un diccionari dels $m$ usuaris més propers i el seu *score*, que seran els que usarem per fer la recomanació. Normalitzeu els *scores* de sortida.

In [276]:
def find_similar_users(DataFrame, userID, m, simfunction):
    """
    Retorna un diccionari de usuaris similars amb les scores corresponents.
    
    :param DataFrame: dataframe que conté totes les dades
    :param userID: usuari respecte al qual fem la recomanació
    :param m: nombre d'usuaris que volem per fer la recomanació
    :param similarity: mesura de similitud
    :return : dictionary
    """
    # la vostra solució aquí
    # agafem tots els usuaris
    users = DataFrame['user_id'].unique()
    # Per evitar comparar l'usuari respecte al qual fem la recomanació, eliminem aquest de la taula users
    users = np.delete(users,np.where(users == userID))

    # Calculem les puntuacions de similitud amb cada usuari
    similarities = np.array([simfunction(data, userID, user) for user in users])
    # Normalitzem les puntuacions
    normalized_similarities = similarities / similarities.max()

    # Obtenim els m usuaris més semblants i les seves puntuacions normalitzades
    top_similar_users = dict(zip(users, normalized_similarities))
    top_similar_users = dict(sorted(top_similar_users.items(), key=lambda x: x[1], reverse=True)[:m])

    return top_similar_users

In [282]:
t = datetime.datetime.now()
sim_dict = find_similar_users(data, 2, 10, SimEuclid)
t = datetime.datetime.now()-t
print(str(t))

0:00:29.161407


In [283]:
sim_dict

{4447: 1.0,
 1646: 0.9889997267654959,
 5830: 0.9299428385089771,
 2270: 0.9113610085905766,
 1903: 0.9094350911298675,
 3271: 0.9039209357006153,
 4276: 0.9038769023500475,
 1879: 0.8865182530712682,
 3625: 0.8846049354012387,
 650: 0.8825141736204608}

+ Creieu que el temps de procés és assumible? Quan trigaria si ho fem per tots els usuaris?

> **RESPOSTA**: Realment no es gaire assumible ja que per trobar m usuaris similars de nomes un usuari esta trigant entre 19 i 22 segons. Se fessim aixo per tots els usuaris de la base de dades, trigaria aproximadament 37-38 hores lo cual no es una bona mesura.

+ Per solucionar el problema anterior, construeix una matriu de mida $U \times U$ on cada posició $(i,j)$ indiqui la distància entre l'element $i$ i el $j$. Així doncs, si estàs fent un recomanador basat en usuaris, `matriu[2, 3]` contindrà la similitud entre l'usuari 2 i el 3. Calcula quant triga la teva implementació. 

Heu de tenir en compte que:

* Si feu una funció que treballi amb els vectors de cada usuari i faci un doble ``for``, el procés de les dades pot trigar una bona estona.
* Si feu una funció que treballi específicament amb matrius (i no vectors) trigarà molt pocs segons. En aquest link podeu trobar indicacions de com fer-ho: https://jaykmody.com/blog/distance-matrices-with-numpy/



In [191]:
def compute_distance(fixed_arr, var_arr):
    """
    Donats dos vectors, calcula la distancia entre els subvectors formats 
    pels elements en comú. 
    """
    
    # la vostra solució aquí

    mask = np.power(fixed_arr, 2) == np.power(fixed_arr, 2)
    mask &= np.power(var_arr, 2) == np.power(var_arr, 2)

    fixed_arr_common = fixed_arr[mask]
    var_arr_common = var_arr[mask]

    return np.linalg.norm(fixed_arr_common - var_arr_common)
    
    
    

In [250]:
def similarity_matrix(compute_distance, df_counts):
    """
    Retorna una matriu de mida M x M on cada posició 
    indica la similitud entre usuaris (resp. ítems).
    
    :param df_counts: df amb els valor que cada usuari li ha donat a una peli.
    :return : Matriu numpy de mida M x M amb les similituds.
    """
    # extraiem valors del dataframe en una matriu
    data_mat = df_counts.values
    ## m = len(data_mat)
    # Como dataframe te valors Nan, els treim sustituint per un 0 per evitar problemes de calcul de similitud
    arr = np.nan_to_num(data_mat)
    # calculem la norma euclidiana
    norma_euclidiana = np.sum(arr**2, axis=1).reshape(-1, 1)
    # Multiipliquem la matriu èr la trasposta per crear la matriu de similitud
    similitud_matrix = np.dot(arr, arr.T)
    # Ara fent servir la norma euclidiana calculem la distancia euclidiana que repreentara la similitud que hi ha entre ususaris
    distancia_eucli = np.sqrt(norma_euclidiana + norma_euclidiana.T - 2 * similitud_matrix)
    # Tenim en compte el nombre de pelicules en comú
    mask = np.where(arr != 0, 1,0)
    common_movies = np.matmul(mask, (mask.T/ arr.shape[1]))
    normalized_sim_mtx = common_movies*1 / (1 + distancia_eucli)
    return normalized_sim_mtx


In [251]:
t = datetime.datetime.now()
# Aquest subset l'he creat per fer proves amb altres implementacion que al ser ineficients, no arriba memoria sificient per totes les dades
#subset = df_counts.sample(n=100)
sim = similarity_matrix(compute_distance, df_counts)
t = datetime.datetime.now()-t
print("Temps amb doble for:",str(t))
print(sim)

Temps amb doble for: 0:00:07.782452
[[1.43011333e-02 3.63236332e-05 3.99160008e-05 ... 0.00000000e+00
  8.10159742e-05 6.26354958e-05]
 [3.63236332e-05 3.48084188e-02 6.54173950e-05 ... 1.72193823e-05
  3.55436244e-05 1.69970264e-04]
 [3.99160008e-05 6.54173950e-05 1.37614679e-02 ... 2.46354581e-05
  4.23101097e-05 7.43023524e-05]
 ...
 [0.00000000e+00 1.72193823e-05 2.46354581e-05 ... 5.39665407e-03
  4.15460809e-05 4.17798553e-05]
 [8.10159742e-05 3.55436244e-05 4.23101097e-05 ... 4.15460809e-05
  3.31894226e-02 1.49033643e-04]
 [6.26354958e-05 1.69970264e-04 7.43023524e-05 ... 4.17798553e-05
  1.49033643e-04 9.20129520e-02]]


+ Ara torna a re-fer la funció ``find_similar_users`` i mira quant triga. 

> Recorda que les scores han d'estar normalitzades!

In [194]:
def find_similar_users(DataFrame, sim_mx, userID, m):
    # la vostra solució aquí

    # Agafem tots els usuaris
    users = DataFrame['user_id'].unique()
    # Per evitar comparar l'usuari respecte al qual fem la recomanació, eliminem aquest de la taula users
    users = np.delete(users,np.where(users == userID))
    
    # Agafem l'array de les similituds a l'usuari
    data = sim_mx[userID]

    # Normalitzem les dades
    normalized_similarities = data / data.max()
    
    # Obtenim els m usuaris més semblants i les seves puntuacions normalitzades
    top_similar_users = dict(zip(users, normalized_similarities))
    top_similar_users = dict(sorted(top_similar_users.items(), key=lambda x: x[1], reverse=True)[:m])
    
    return top_similar_users

    

In [195]:
t = datetime.datetime.now()
sim_dict = find_similar_users(data, sim, 2, 10)
t = datetime.datetime.now()-t
print(str(t))

0:00:00.062035


In [196]:
sim_dict

{11: 1.0,
 5991: 0.012366322972834095,
 2649: 0.012063935950732255,
 1757: 0.01184086000176346,
 5862: 0.011760564122430843,
 2545: 0.011489329635531254,
 5027: 0.01130900340974312,
 3992: 0.011257345507579166,
 2571: 0.011057255373790728,
 4778: 0.010984960903376216}

#### EXERCICI F.2

+ Computa les recomanacions per un usuari concret a partir dels ratings dels seus $m$ usuaris més propers. Fes primer una funció que retorni la **weighted average list** dels $m$ usuaris més propers. Feu servir la funció anterior que usava la matriu de similituds per anar més ràpid!! (Nota: la **weighted average list** es calcularà agregant els $n$ items més puntuats de cadascun dels m users més semblants al usuari donat).

In [353]:
def weighted_average(DataFrame, user, sim_mx, m):
    """
    df_counts: usuaris_semblants * pelis
    similarity: {user: similarity}
    
    Aquesta funcio calculara el score predit (Score que ha donat l'usuariB * similitud(A,B)) i despres sumara
    tots els valors per cada peli; retornara {peli_id: suma de score predit}
    """
    # la vostra solució aquí

    # Encuentra los m usuarios más similares
    similar_users = np.argsort(sim_mx[user])[-m:]
    
    # Calcula el promedio ponderado para cada película
    weighted_avg = {i: 0 for i in range(1, DataFrame.nunique()['movie_id']+1)}
    
    for i in range(1, DataFrame.nunique()['movie_id'] +1): 
        Simsuma = 0
        for other_user in similar_users:    
            data_user = DataFrame[DataFrame['user_id'] == other_user]
            if not data_user[data_user['movie_id'] == i]['rating'].isna().all():
                weighted_avg[i] += sim_mx[user, other_user] * data_user.loc[data_user['movie_id'] == i, 'rating'].values[0]
                Simsuma += sim_mx[user, other_user]
        if weighted_avg[i] != 0:
            weighted_avg[i] /= Simsuma
    return weighted_avg


In [368]:
def getRecommendationsUser(DataFrame, user, sim_mx, n, m):
    """
    Retorna un dataframe de pel·licules amb els scores.
    
    :param DataFrame: dataframe que conté totes les dades
    :param user: usuari al qual fem la recomanació
    :param sim_mx: similarity_function
    :param n: nombre de pelis a recomanar
    :param m: nombre d'usuaris semblants a tenir en compte per les recomanacions
    :return : pandas de pelis amb els seus scores predits
    """
    
    # la vostra solució aquí
    
    # Calcula el promedio ponderado para cada película
    weighted_avg = weighted_average(DataFrame, user, sim_mx, m)
   
    top_x_elements = sorted(weighted_avg.items(), key=lambda item: item[1], reverse=True)[:n]

    df = pd.DataFrame(top_x_elements, columns=['mavie_id', 'rating'])
    return df
    

In [370]:
t = datetime.datetime.now()
user_prediction = getRecommendationsUser(data, 271, sim, 10, 50)
t = datetime.datetime.now()-t
print(str(t))

In [367]:
user_prediction

,mavie_id,rating
0,34,0
1,39,0
2,48,0
3,51,0
4,54,0
5,65,0
6,78,0
7,81,0
8,82,0
9,93,0


### 3.7. EXERCICI G


A continuació usarem la metrica **Mean Absolute Error (MAE)** per evaluar el nostre sistema. Aquesta mètrica ens permetrà mesurar la diferencia entre dues llistes donat un usuari: 
+ La llista amb els ratings originals d'un usuari donat
+ La llista de les prediccions generades per aquest usuari

#### EXERCICI G.1

+ Treu el 10% dels usuaris i reserva aquests en una variable anomenada ``test_set`` i la resta en una variable anomenada ``train_set``.

In [295]:
# la vostra solució aquí
test_set = data.head(int(len(data)*(10/100)))
train_set = data.tail(len(data) - len(test_set))

assert len(test_set) + len(train_set) == len(data)

+ Què passarà si calculo la matriu de similitud amb ``train_set`` i després intento predir pels usuaris de ``test_set``??

> **Resposta**:


#### EXERCICI G.2

+ Selecciona aproximadament el 80% de les interaccions de cada usuari de test i junta-les al ``train_set``. Podem ara podem evaluar el sistema?


> Com la pràctica és molt llarga us donem el codi per un usuari donat i vosaltres només heu de crear la funció que, per cada usuari, afagi el 80% de les intraccions i les unifiqui al dataframe de train.

In [296]:
test_set.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,0,1104,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,1104,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,11,1104,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,14,1104,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,16,1104,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama


In [297]:
# Agafem el 20% de les pelis que ha consumit cada usuari de test 
groupby_count = test_set.groupby('user_id')['movie_id'].count()*0.2
groupby_count

user_id
0       10.6
1        9.8
2        1.8
3        1.2
4        3.0
        ... 
6035    11.8
6036     4.2
6037     0.2
6038     3.6
6039     5.6
Name: movie_id, Length: 6002, dtype: float64

Seleccionem la posició 1 i aquest use_id serà el que usarem pel codi d'exemple (que després haureu de replicar).

In [298]:
groupby_count.reset_index().iloc[1]

user_id     1.0
movie_id    9.8
Name: 1, dtype: float64

In [299]:
n_test_samples = int(groupby_count.reset_index().iloc[1]['movie_id'])
u = groupby_count.reset_index().iloc[1]['user_id']

In [300]:
test_set_user = test_set[test_set['user_id'] == u]
frame_test = test_set_user.sample(n_test_samples)
print("TOTAL SAMPLES OF THE USER: " + str(len(test_set_user)))
print("TOTAL SAMPLES OF THE USER IN TEST SET: " + str(len(frame_test)))

TOTAL SAMPLES OF THE USER: 49
TOTAL SAMPLES OF THE USER IN TEST SET: 9


In [301]:
len(test_set_user.index)

49

In [302]:
frame_train = test_set_user[~test_set_user.index.isin(frame_test.index)]
print("TOTAL SAMPLES OF THE USER IN TRAIN SET: " + str(len(frame_train)))

TOTAL SAMPLES OF THE USER IN TRAIN SET: 40


In [303]:
assert len(frame_train) + len(frame_test) == len(test_set_user)

In [308]:
def add_testdata(traindf, test_set):
    """
    Retorna els N usuaris més similars basat en la correlació de Pearson (no)
    
    :param traindf: dataframe que conté les dades de train
    :param test_set: dataframe que conté les dades de test

    :return : 
        - :param 1st: dataframe que conté les dades de train juntament amb el 80% de test seleccionat
        - :param 2nd: dataframe que conté les dades de test que queden (20% restant)
    """
    
    # la vostra solució aquí    
    
    groupby_count = test_set.groupby('user_id')['movie_id'].count()*0.2

    # agafem tots els usuaris
    users = groupby_count.reset_index().index

    train = traindf
    test =  pd.DataFrame()

    for user in users:
        n_test_samples = int(groupby_count.reset_index().iloc[user]['movie_id'])
        u = groupby_count.reset_index().iloc[user]['user_id']
        test_set_user = test_set[test_set['user_id'] == u]
        frame_test = test_set_user.sample(n_test_samples)
        frame_train = test_set_user[~test_set_user.index.isin(frame_test.index)]


        train = pd.concat([frame_train, train], ignore_index=True)
        test = pd.concat([frame_test,test], ignore_index=True)

    return train, test



In [ ]:
train, test = add_testdata(train_set, test_set)

In [309]:
train

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,6039,1104,4,957716612,M,25,6,11106,One Flew Over the Cuckoo's Nest (1975),Drama
1,6039,858,5,956704191,M,25,6,11106,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical
2,6039,2586,4,956715569,M,25,6,11106,Airplane! (1980),Comedy
3,6039,1178,3,997453960,M,25,6,11106,Back to the Future (1985),Comedy|Sci-Fi
4,6039,513,5,956704219,M,25,6,11106,Schindler's List (1993),Drama|War
...,...,...,...,...,...,...,...,...,...,...
982629,5948,2017,5,958846401,M,18,17,47901,Modulations (1998),Documentary
982630,5674,2498,3,976029116,M,35,14,30030,Broken Vessels (1998),Drama
982631,5779,2638,1,958153068,M,18,17,92886,White Boys (1999),Drama
982632,5850,3367,5,957756608,F,18,20,55410,One Little Indian (1973),Comedy|Drama|Western


In [310]:
test

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,6039,1154,3,956715834,M,25,6,11106,Dead Poets Society (1989),Drama
1,6039,1781,4,956703977,M,25,6,11106,Rain Man (1988),Drama
2,6039,1167,3,956715421,M,25,6,11106,Stand by Me (1986),Adventure|Comedy|Drama
3,6039,689,4,957717191,M,25,6,11106,Wallace & Gromit: The Best of Aardman Animatio...,Animation
4,6039,144,3,956704716,M,25,6,11106,Apollo 13 (1995),Drama
...,...,...,...,...,...,...,...,...,...,...
17570,0,1727,4,978824330,F,1,10,48067,Mulan (1998),Animation|Children's
17571,0,2710,4,978302124,F,1,10,48067,Ferris Bueller's Day Off (1986),Comedy
17572,0,858,4,978301368,F,1,10,48067,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical
17573,0,2898,4,978302174,F,1,10,48067,Toy Story 2 (1999),Animation|Children's|Comedy


In [311]:
train.shape

(982634, 10)

In [312]:
test.shape

(17575, 10)

In [313]:
data.shape

(1000209, 10)

In [314]:
assert train.shape[0] + test.shape[0] == data.shape[0]

#### EXERCICI G.3

+ Fes una funció que serveixi per evaluar el nostre sistema usant la mètrica MAE. 

In [338]:
def evaluateRecommendations(train, test, m,n, sim):
    """
    Retorna l'error generat pel model
    
    :param DataFrame: dataframe que conté totes les dades
    :param userID: usuari respecte al qual fem la recomanació
    :param m: nombre d'usuaris que volem per fer la recomanació
    :param n: nombre de pelis a retornar (no)
    :param sim: matriu de similitud
    :return : Escalar (float) corresponent al MAE
    """
   
    # la vostra solució 
    users = test['user_id'].unique()
    mae = 0
    test = test.sort_values(by='user_id')
    for user in users:
        weighted_avg = weighted_average(train, user, sim, m)
        
    

In [339]:
t = datetime.datetime.now()
mae = evaluateRecommendations(train, test, 50, 10, sim)
t = datetime.datetime.now()-t
print(str(t))

6002
0:00:00.005313


In [ ]:
mae

### 3.8. EXERCICI H (exercici opcional, no obligatori)


+ **Que surt més a compte, fer un recomanador unic pels dos sexes o un per cada sexe?** Justifica la resposta per escrit i amb el codi necessari.